In [ ]:
import os
import numpy as np
import librosa
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, Concatenate, Average, Flatten, Dropout, Bidirectional, LSTM, GRU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FastICA
from scipy.spatial.distance import cdist
import tensorflow as tf
from sklearn.metrics import accuracy_score

# Paths to the datasets
angry_path = os.path.expanduser('~/Documents/notebook/MMNet/Dataset/Anger')
calm_path = os.path.expanduser('~/Documents/notebook/MMNet/Dataset/Calm')
neutral_path = os.path.expanduser('~/Documents/notebook/MMNet/Dataset/Neutral')
happy_path = os.path.expanduser('~/Documents/notebook/MMNet/Dataset/Happy')
sad_path = os.path.expanduser('~/Documents/notebook/MMNet/Dataset/Sad')

# Define the number of MFCCs (Mel Frequency Cepstral Coefficients) to extract from each audio file
num_mfcc = 40
max_len = 500

# Define a function to extract the MFCCs from an audio file with noise filtering
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=44100)
    y = librosa.effects.preemphasis(y)  # Apply preemphasis for noise filtering
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=num_mfcc)
    return mfccs

# Load the speech data
X = []
y = []
for path, emotion in [(angry_path, 0), (calm_path, 1), (neutral_path, 2), (happy_path, 3), (sad_path, 4)]:
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        mfccs = extract_features(file_path)
        if len(mfccs.T) <= max_len:
            pad_width = max_len - len(mfccs.T)
            mfccs = np.pad(mfccs.T, pad_width=((0, pad_width), (0, 0)), mode='constant')
            X.append(mfccs)
            y.append(emotion)

X = np.array(X)
y = np.array(y)

# Convert the target labels to one-hot encoded vectors
y = to_categorical(y)

# Load EEG data
path_csv = os.path.expanduser('~/Documents/notebook/MMNet/Dataset/emotions.csv')
EEGData = pd.read_csv(path_csv)

# EEG Data Preprocessing: Noise Filtering, Artifact Removal, and Normalization
def preprocess_eeg_signal(EEGData):
    # Apply ICA for artifact removal
    ica = FastICA(n_components=3)
    X_eeg = EEGData.drop(["label"], axis=1).values
    X_ica = ica.fit_transform(X_eeg)

    # Normalize the EEG data (Z-score normalization)
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X_ica)

    # Encode labels
    label_mapping = {'NEUTRAL': 0, 'POSITIVE': 1, 'NEGATIVE': 2}
    y = EEGData['label'].map(label_mapping).values
    y = to_categorical(y)

    return X_normalized, y

# Preprocess EEG data
X_eeg, Y_eeg = preprocess_eeg_signal(EEGData)

# Split the EEG data into training and testing sets
x_train_eeg, x_test_eeg, y_train_eeg, y_test_eeg = train_test_split(X_eeg, Y_eeg, test_size=0.2, random_state=4)

# Trim the larger dataset to match the smaller dataset
min_samples = min(X.shape[0], X_eeg.shape[0])
X_trimmed = X[:min_samples]
x_train_trimmed = x_train_eeg[:min_samples]
y_train_trimmed = y[:min_samples]
y_train_eeg_trimmed = y_train_eeg[:min_samples]

# Reshape the speech data
X_trimmed = np.expand_dims(X_trimmed, axis=-1)

# Define the speech model (CNN-BiLSTM)
def create_speech_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(max_len, num_mfcc, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(TimeDistributed(Flatten()))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Bidirectional(LSTM(units=128)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))  # 5 classes for speech
    return model

# Define the EEG model (BiGRU)
def create_eeg_model():
    inputs = Input(shape=(x_train_eeg.shape[1],))
    expand_dims = tf.expand_dims(inputs, axis=2)
    gru = Bidirectional(GRU(64, return_sequences=True))(expand_dims)
    gru = BatchNormalization()(gru)
    gru = Dropout(0.3)(gru)
    gru = Bidirectional(GRU(128, return_sequences=True))(gru)
    gru = BatchNormalization()(gru)
    gru = Dropout(0.3)(gru)
    gru = Bidirectional(GRU(64, return_sequences=False))(gru)
    gru = BatchNormalization()(gru)
    gru = Dropout(0.3)(gru)
    flatten = Flatten()(gru)
    dense = Dense(128, activation='relu')(flatten)
    dense = Dropout(0.3)(dense)
    outputs = Dense(5, activation='softmax')(dense)  # 3 classes for EEG
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define DTW function for dynamic time warping between the two models' feature vectors
from dtw import accelerated_dtw

# Function to align each speech and EEG vector pair using DTW
def align_with_dtw(speech_feats, eeg_feats):
    aligned = []
    for i in range(len(speech_feats)):
        d, cost_matrix, acc_cost_matrix, path = accelerated_dtw(
            speech_feats[i].reshape(-1, 1),
            eeg_feats[i].reshape(-1, 1),
            dist='euclidean'
        )
        # Align both vectors to the same length (e.g., average path or interpolate)
        aligned_speech = speech_feats[i][path[0]]
        aligned_eeg = eeg_feats[i][path[1]]
        # Average aligned vectors (or concatenate, depending on your goal)
        fused = np.concatenate([aligned_speech, aligned_eeg], axis=0)  # shape: (aligned_len * 2,)
        aligned.append(fused)
    return np.array(aligned)

# Combine the CNN-BiLSTM and BiGRU models' output feature vectors using DTW
speech_model = create_speech_model()
eeg_model = create_eeg_model()

# Compile both models
speech_model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
eeg_model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Train both models
speech_model.fit(X_trimmed, y_train_trimmed, validation_split=0.2, epochs=2, batch_size=16)
eeg_model.fit(x_train_trimmed, y_train_eeg_trimmed, validation_split=0.2, epochs=2, batch_size=16)

# Get feature vectors from both models
speech_output = speech_model.predict(X_trimmed)
eeg_output = eeg_model.predict(x_train_trimmed)

# Apply Dynamic Time Warping (DTW) to combine the outputs
dtw_distance = align_with_dtw(speech_output, eeg_output)

combined_output = dtw_distance

from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import numpy as np


# Reshape for Transformer input
fused_features = np.reshape(combined_output, (100, 2, 128))

# Define Transformer Fusion Model
def transformer_fusion_model(input_shape):
    inputs = Input(shape=input_shape)
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    x = Dropout(0.1)(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(5, activation='softmax')(x)  # Assuming 5-class output
    return Model(inputs, outputs)

# Compile the model
fusion_model = transformer_fusion_model((2, 128))
fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Dummy labels for demonstration
y_dummy = to_categorical(np.random.randint(0, 5, size=(100,)), num_classes=5)

# Train
fusion_model.fit(fused_features, y_dummy, epochs=25, batch_size=16, validation_split=0.2)

# Predict and evaluate
preds = fusion_model.predict(fused_features)
pred_labels = np.argmax(preds, axis=1)
true_labels = np.argmax(y_dummy, axis=1)

from sklearn.metrics import accuracy_score
acc = accuracy_score(true_labels, pred_labels)
print(f"Accuracy: {acc:.4f}")
